In [2]:
import pandas as pd
import bs4
#import selenium
import requests
import re

In [35]:
url = "http://localhost:8000/students.html"
page = requests.get(url).text
soup = bs4.BeautifulSoup(page)
soup

<html>
<head>
<title>Web scraping</title>
</head>
<body>
<div class="student-details">
<p class="gl-jan" data-gender="f" id="student-10"> ABC-1 </p>
<p class="gl-jan" data-gender="m" id="student-11"> ABC-2 </p>
<p class="gl-jan" data-gender="f" id="student-12"> ABC-3 </p>
<p class="gl-jan" data-gender="m" id="student-13"> ABC-4 </p>
</div>
</body>
</html>

In [36]:
print(soup.title)

<title>Web scraping</title>


In [37]:
soup.title.text

'Web scraping'

In [39]:
soup.findAll('p')

[<p class="gl-jan" data-gender="f" id="student-10"> ABC-1 </p>,
 <p class="gl-jan" data-gender="m" id="student-11"> ABC-2 </p>,
 <p class="gl-jan" data-gender="f" id="student-12"> ABC-3 </p>,
 <p class="gl-jan" data-gender="m" id="student-13"> ABC-4 </p>]

In [41]:
tags = soup.findAll('p')
df = pd.DataFrame()
for tag in tags:
    name = tag.text
    student_id = tag.get('id')
    gender = tag.get('data-gender')
    temp = {
        'name': name,
        'student_id': student_id,
        'gender': gender,
    }
    df = df.append(temp, ignore_index=True)
df

,gender,name,student_id
0,f,ABC-1,student-10
1,m,ABC-2,student-11
2,f,ABC-3,student-12
3,m,ABC-4,student-13


In [44]:
soup.findAll('p', {'class': 'gl-jan'})

[<p class="gl-jan" data-gender="f" id="student-10"> ABC-1 </p>,
 <p class="gl-jan" data-gender="m" id="student-11"> ABC-2 </p>,
 <p class="gl-jan" data-gender="f" id="student-12"> ABC-3 </p>,
 <p class="gl-jan" data-gender="m" id="student-13"> ABC-4 </p>]

## Scraping wikipedia paragraphs

In [3]:
url = 'https://en.wikipedia.org/wiki/Data_science'
page = requests.get(url).text
soup = bs4.BeautifulSoup(page)

In [49]:
soup.title.text

'Data science - Wikipedia'

In [58]:
para_tags = soup.findAll('p')

para_texts = []
for tag in para_tags:
    para_texts.append(tag.text)
article = ' '.join(para_texts)
article = article.replace('\n','')
len(article)

6864

In [65]:
tables = soup.findAll('table')

In [68]:
df_tables = pd.read_html(url)
df_tables[1]

,vteData,vteData.1
0,Augmentation Analysis Archaeology Cleansing Co...,Augmentation Analysis Archaeology Cleansing Co...


In [4]:
!pip install html5lib

In [5]:
url = 'https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory'
tables = pd.read_html(url, flavor='html5lib')

In [77]:
df_cases = tables[6]

In [78]:
df_cases.to_excel('corona-cases.xlsx', sheet_name='country-cases')

In [ ]:
#!pip install xlsxwriter
#writer = pd.ExcelWriter('corona-cases.xlsx', engine='xlswriter')
#df_cases.to_excel(writer, sheet_name="country-cases")

## Naukri jobs

In [8]:
url = 'https://www.naukri.com/data-science-jobs?k=data%20science'

HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

page = requests.get(url, headers=HEADERS).text